In [ ]:
from dgp import *

# 1. Simulate or load data
sample_size = 800
min_degree = 1
max_degree = 2
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=500,
    burnin_R=20, seed=0)

Y_chain = Y_chain[::5]
A_chain = A_chain[::5]
L_chain = L_chain[::5]

100%|██████████| 520/520 [00:42<00:00, 12.15it/s]


In [ ]:
n_cpu = 10

In [2]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=200,
    burnin_R=50,
    seed=0
)

{'average': 0.31335625,
 'direct_effect': -0.14082500000000003,
 'spillover_effect': -0.05294999999999994,
 'psi_1_gamma': 0.27003750000000004,
 'psi_0_gamma': 0.4108625000000001,
 'psi_zero': 0.4638125}

In [ ]:
from utils import run_pll
from run_pll import run_dr

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_{sample_size}_{min_degree}_{max_degree}.npy', res_list_array_dr)

In [ ]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [ ]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)
# save results
np.save(f'run/sim_results/sim_autog_{sample_size}_{min_degree}_{max_degree}.npy', res_list_array_ag)

In [ ]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")